In [9]:
## Create constants and import hash libraries

# Prime order of curve
P = 2**256 - 2**32 - 977
# The order of the base point (also known as the generator point G),
# which is the number of distinct points on the curve that can be generated by
# repeatedly adding the base point to itself.
Q = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
# Generator point Cordinates
G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8


CONTEXT = b'FROST-BIP340'


In [10]:

# Define ECC point class
class Point:
        """Class representing an elliptic curve point."""
        def __init__(self, x=float('inf'), y=float('inf')):
            self.x = x
            self.y = y

        @classmethod
        def secret_deserialize(cls, hex_public_key):
            P = P
            hex_bytes = bytes.fromhex(hex_public_key)
            is_even = hex_bytes[0] == 2
            x_bytes = hex_bytes[1:]
            x = int.from_bytes(x_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 == 0:
                even_y = y
                odd_y = (P - y) % P
            else:
                even_y = (P - y) % P
                odd_y = y
            y = even_y if is_even else odd_y

            return cls(x, y)

        def secret_serialize(self):
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'

            return prefix + self.x.to_bytes(32, 'big')

        @classmethod
        def xonly_deserialize(cls, hex_public_key):
            P = FROST.secp256k1.P
            hex_bytes = bytes.fromhex(hex_public_key)
            x = int.from_bytes(hex_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 != 0:
                y = (P - y) % P

            return cls(x, y)

        def xonly_serialize(self):
            return self.x.to_bytes(32, 'big')

        # point at infinity
        def is_zero(self):
            return self.x == float('inf') or self.y == float('inf')

        def __eq__(self, other):
            return self.x == other.x and self.y == other.y

        def __ne__(self, other):
            return not self == other

        def __neg__(self):
            if self.is_zero():
                return self

            return self.__class__(self.x, P - self.y)

        # Double point
        def dbl(self):
            x = self.x
            y = self.y
            s = (3 * x * x * pow(2 * y, P - 2, P)) % P
            sum_x = (s * s - 2 * x) % P
            sum_y = (s * (x - sum_x) - y) % P

            return self.__class__(sum_x, sum_y)

        def __add__(self, other):
            if self == other:
                return self.dbl()
            if self.is_zero():
                return other
            if other.is_zero():
                return self
            if self.x == other.x and self.y != other.y:
                return self.__class__()
            s = ((other.y - self.y) * pow(other.x - self.x, P - 2, P)) % P
            sum_x = (s * s - self.x - other.x) % P
            sum_y = (s * (self.x - sum_x) - self.y) % P

            return self.__class__(sum_x, sum_y)

        def __rmul__(self, scalar):
            p = self
            r = self.__class__()
            i = 1

            while i <= scalar:
                if i & scalar:
                    r = r + p
                p = p.dbl()
                i <<= 1

            return r

        def __str__(self):
            if self.is_zero():
                return '0'
            return 'X: 0x{:x}\nY: 0x{:x}'.format(self.x, self.y)

        def __repr__(self) -> str:
            return self.__str__()

In [11]:
# Create ECC Generator point
G = Point(G_x, G_y)


## DKG
Perdensen key generation is a two step process
1. Generate and distribute public shares, polynomial coefficients, and commitments to the shares
2. Participants sign for their commitment other participants verify


TODO why is this important


In [15]:
import secrets
from hashlib import sha256

# Lets create a 2:3 multisig
THRESHOLD = 2
N = 3

def new_polynomial(threshold):
    # 1. Generate Shamir polynomial with random coefficients, and with degree
    # equal to the threshold minus one.
    # (a_i_0, . . ., a_i_(t - 1)) ⭠ $ ℤ_q
    coefficients = [secrets.randbits(256) % Q for _ in range(threshold)]
    coefficient_points = [coefficient * G for coefficient in coefficients]
    return coefficient_points

def proof_of_knowledge(index, ):
    # Use a new nonce to avoid signature replay attack
    # k ⭠ ℤ_q
    nonce = secrets.randbits(256) % Q
    # R = g^k
    nonce_commitment = nonce * G
    # x_0 is always the partial private key
    secret = coefficients[0]
    secret_commitment = secret * G

    # c = H(i, 𝚽, s, R)
    # Commit to the your position in signing session
    # The frost tag
    # Your partial secret key
    # Your public partial nonce
    challenge_hash = sha256()
    challenge_hash.update(index_bytes = index.to_bytes(1, 'big'))
    challenge_hash.update(CONTEXT)
    challenge_hash.update(secret_commitment.secret_serialize())
    challenge_hash.update(nonce_commitment.secret_serialize())
    challenge_hash_bytes = challenge_hash.digest()
    challenge_hash_int = int.from_bytes(challenge_hash_bytes, 'big')
    # s_i = k + f(0) * c
    s = (nonce + secret * challenge_hash_int) % Q
    proof_of_knowledge = [nonce_commitment, s]
    
    return proof_of_knowledge


In [ ]:
# We want to verify a participant verifies knowledge of their coeffcients

def verify_proof_of_knowledge(index, proof, secret_commitment):
    pass
    